In [1]:
%matplotlib inline

from consts import *
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
from keras.models import Model
from joblib import dump, load
import keras
import tensorflow as tf
import json

dest_dir = base_path + 'images/rcnn_out/'

USE_SVM = False

ss_info_filepath = base_path + 'ss_info.csv'


In [2]:
# ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
# ss_file = open(ss_info_filepath, "a+")

# for j in range(33,38):
#     img_path = base_path + 'images/rcnn_test/test' + str(j) + '.jpg'
#     # run selective search
#     image = cv2.imread(img_path)[...,::-1] # switch to rgb

#     # initialise selective search
#     ss.setBaseImage(image)
#     ss.switchToSelectiveSearchFast()
#     ssresults = ss.process()
#     print(ssresults.shape)
    
#     ss_dict = {'results': np.ndarray.tolist(ssresults)[:2000]}
#     ss_file.write(img_path + ',' + '\"' + json.dumps(ss_dict) + '\"' + '\n')

# ss_file.close()

In [3]:
images = []
all_ssresults = []
file_names = []
df = pd.read_csv(ss_info_filepath)

for index, row in df.iterrows():
    img_path = row['path'].strip()
    if not os.path.isfile(img_path):
        continue
    ss_results = json.loads(row['ss_results'])['results']
    image = cv2.imread(img_path)[...,::-1] # switch to rgb
    images.append(image)
    all_ssresults.append(ss_results)
    file_names.append(img_path.split('/')[-1])

In [4]:
# load up the CNN weights
weights_dir = base_path + 'weights/' + 'cnn_tuned_checkpoint.h5'

from CNN_model import VGG_model, custom_model

if USE_SVM:
    # model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
    custom_model.load_weights(weights_dir + 'cnn_0_120_good.h5')
    model = Model(inputs=model.input,
                       outputs=model.layers[-2].output)
else:
    VGG_model.load_weights(weights_dir)
    model = VGG_model


Num GPUs Available:  1


In [ ]:
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)
    
def resize_norm(crop):
    resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
    if not USE_GREY:
        resized_norm = (resized/255) - 0.5
    else:
        resized_norm = resized - 0.5
    resized_norm = np.expand_dims(resized_norm, axis=0)
    return resized_norm

def limit_size(x, y, w, h):
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if x + w >= im_w:
        w = im_w - x - 1
    if y + h >= im_h:
        h = im_h - y - 1
    return (x, y, w, h)

# TODO: Fix for USE_GREY
MAX_SIZE = 650
j = 0
n = 0
inc = 0

for fname, ssresults, image in zip(file_names, all_ssresults, images):
    print(fname)
    if USE_GREY:
        imout = io.imread(img_path, as_gray=True)
        imout = np.expand_dims(imout, axis=2)
    else:
        imout = image

    crop_list = []

    for i,result in enumerate(ssresults):
        if i >= 2000:
            break
        x,y,w,h = result

        if w < MIN_SIZE or h < MIN_SIZE or h > MAX_SIZE or w > MAX_SIZE:
            continue
        rect = Rectangle([x ,y], w, h, fill=False, 
                                color='green', lw=0.5)
    #         ax.add_patch(rect)

        crop = imout[y:y+h,x:x+w]
        resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)

        if not USE_GREY:
            resized_norm = (resized/255) - 0.5
        else:
            resized_norm = resized - 0.5

        resized_norm = np.expand_dims(resized_norm, axis=0)

        if USE_SVM:
            layer_output = np.array(model(resized_norm))
            prob = clf.predict_proba(layer_output)
        else:
            prob = model.predict(resized_norm)
        if prob[0][1] > 0.2:
            crop = [x, y, w, h]
            crop_list.append([crop, prob[0][1], resized])

    im_h, im_w, _ = image.shape 

    plt.figure(figsize=(18,18))
    ax = plt.gca()
    ax.imshow(image)
    sorted_crop_list = list(sorted(crop_list, key=lambda item:item[1], reverse=True))
    print('Number of possible regions:' + str(len(sorted_crop_list)))

    max_k = len(sorted_crop_list)
    if max_k > 15:
        max_k = 15

    colors = ['red', 'orange', 'yellow', 'blue', 'purple', 'pink', 'magenta', 'cyan', 'black', 'white']

    k = 0
    begin_flag = 0
    for crop_img in sorted_crop_list:
        if k >= max_k:
            break
        crop_dims, prob, resized = crop_img
        
        # only consider the first entry < 0.5 probability
        if prob < 0.5 and (k != 0 or begin_flag):
            break
        begin_flag = 1
        
        x, y, w, h = crop_dims
        rect = Rectangle([x ,y], w, h, fill=False, color='green', lw=1)
        ax.add_patch(rect)
        
        probs = []
        
        # test an image slightly smaller
        new_w = int(w*0.95)
        new_h = int(h*0.95)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])    
        
        # test an image even larger
        new_w = int(w*1.05)
        new_h = int(h*1.05)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])
        
        # test an image even larger
        new_w = int(w*1.1)
        new_h = int(h*1.1)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])
        
        # test an image even larger
        new_w = int(w*1.15)
        new_h = int(h*1.15)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])  

        # test an image even larger
        new_w = int(w*1.2)
        new_h = int(h*1.2)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])   

        min_prob = min(probs)
        max_prob = max(probs)
        test_prob = sum(probs) / len(probs) 
        print(prob, min_prob, max_prob, test_prob)
        
        barrier1 = 0.8
        
        if w < 150 and h < 150:
            barrier1 = 0.95
        elif w > 500 or h > 500:
            barrier1 = 0.95
        elif w > 400 or h > 400:
            barrier1 = 0.9
        
        if min_prob < barrier1:
            continue
#             if not (k == 0 and test_prob > 0.7 and max_prob > 0.9):
#                 continue
#             else:
#                 col = 'yellow'
        else:
            col = 'red'
                
        prob_str = str(prob)[0:6]
        prob_str += ', ' + str(min_prob)[0:6]
        
        ax.text(x,y + 3, prob_str, c=col, ha='left', va='top', bbox=dict(facecolor='white', alpha=0.25, lw=0))
        rect = Rectangle([x ,y], w, h, fill=False, 
                            color=col, lw=1)
        ax.add_patch(rect)
        k += 1

    save_file = dest_dir + fname
    j += 1
    print('\n')
    plt.savefig(save_file, bbox_inches='tight')
    plt.close()
    inc += 1

test0.jpg
Number of possible regions:5
0.8704841 0.7360731 0.93652135 0.8197659254074097
0.60840607 0.25649327 0.6786482 0.4072277307510376


test1.jpg
Number of possible regions:7
0.9932447 0.98498803 0.99594647 0.9898472785949707
0.9873143 0.975806 0.9957552 0.9911489605903625
0.89436597 0.9020737 0.9964223 0.9586336493492127


test2.jpg
Number of possible regions:8
0.96092355 0.9586584 0.97373587 0.9650695323944092
0.8066526 0.37639177 0.81309205 0.533050411939621


test3.jpg
Number of possible regions:7
0.8883432 0.8389103 0.9877079 0.9132279515266418
0.6994513 0.5706915 0.8385768 0.7355939388275147
0.5355208 0.284254 0.5403659 0.416716867685318
0.5081713 0.31029835 0.69116515 0.4664174735546112
0.50691605 0.22579725 0.76636106 0.3972670465707779


test4.jpg
Number of possible regions:11
0.9713903 0.973828 0.9891816 0.9820443272590638
0.96904635 0.9166276 0.98841524 0.9722052931785583
0.9651959 0.93704796 0.96962947 0.9521775603294372
0.8611891 0.71844935 0.89027506 0.8262605309486